In [1]:
import os

In [2]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\Projects\\DL\\Workplace-Safety-Detection-CV'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path                      
    data_yaml_file: Path
    best_model: Path

In [6]:
import os
from CV.constants import *
from CV.utils.common import read_yaml, create_directories
from CV.entity.config_entity import EvaluationConfig


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:

        evaluation = self.config.evaluation
        
        create_directories([
            Path(evaluation.root_dir)
        ])

        eval_config = EvaluationConfig(
            root_dir=Path(evaluation.root_dir),
            data_yaml_file=Path(evaluation.data_yaml_file),
            best_model=Path(evaluation.best_model),
        )
        return eval_config

In [14]:
from pathlib import Path
import os
from CV.entity.config_entity import EvaluationConfig
from ultralytics import YOLO


import numpy as np


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def evaluation(self):

        best_model_path = self.config.best_model

        # Load Best model
        try:
            best_model = YOLO(best_model_path)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Best model file not found at {best_model_path}") from e


        # Dynamically resolve the output directory
        base_dir = Path(os.getcwd()).resolve()  # Get the directory of the script
        output_dir = base_dir / self.config.root_dir

        # Ensure the output directory exists
        output_dir.mkdir(parents=True, exist_ok=True)

        # Train the YOLO model
        try:
            # Train the model
            best_model.val(
                data=self.config.data_yaml_file,
                workers=0,  # Avoid multiprocessing issues
                project=str(output_dir),  # Convert Path to string for YOLOv8
                name="yolov8n_evaluation"  # Custom name for this run
            )
        except Exception as e:
                raise e

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
except Exception as e:
   raise e

[2025-01-22 21:42:16,001: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-01-22 21:42:16,021: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 21:42:16,021: INFO: common: created directory at: artifacts]
[2025-01-22 21:42:16,021: INFO: common: created directory at: artifacts\evaluation]
Ultralytics 8.3.65  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\data_ingestion\Workplace-Safety-Dataset\valid\labels.cache... 325 images, 0 backgrounds, 0 corrupt: 100%|██████████| 325/325 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:11<00:00,  1.83it/s]


                   all        325       1509       0.89      0.835      0.917      0.666
                 Boots        245        539      0.927      0.831      0.919      0.622
                Helmet        231        272      0.917      0.808      0.917      0.632
                Person        245        301      0.851      0.833      0.918      0.749
                  Vest        306        397      0.865      0.869      0.913      0.659
Speed: 0.6ms preprocess, 9.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to D:\DS\Projects\DL\Workplace-Safety-Detection-CV\artifacts\evaluation\yolov8n_evaluation2
